In [1]:
from src.data import player_data
from src.util import characters, display_progress
from src.transfer import transfer_model
from tensorflow import keras
import numpy as np
import pickle
import os

# Model with Pre-Trained Base

In [21]:
model = transfer_model(
    name = 'transfer_model', 
    base_model = 'models/SSBML-Base-Model'
);

In [3]:
model.summary()

Model: "transfer_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SSBML-Base-Model (Sequential (None, 512)               6537842   
_________________________________________________________________
head_densex2 (Sequential)    (None, 1)                 83329     
Total params: 6,621,171
Trainable params: 82,817
Non-trainable params: 6,538,354
_________________________________________________________________


# The Players

In [22]:
!ls data/player

Blynde	CuckDaddy  Lie0x  TCBL	gh0st  ixwonkr


In [25]:
# how many clips does a player have?
!ls data/player/Blynde/train | wc -l

4286


# The Player Data

Player Data can be found in data/player/\<player name\>

Nonplayer Data is taken from the large dataset data/character

In [4]:
# name of the player we want to train/test on
player_name = 'Blynde'

player_dir = os.path.join('data/player', player_name)
player_train_dir = os.path.join(player_dir, 'train')
player_test_dir = os.path.join(player_dir, 'test')

nonplayer_dir = 'data/character'
nonplayer_train_dir = os.path.join(nonplayer_dir, 'train')
nonplayer_test_dir = os.path.join(nonplayer_dir, 'test')

In [5]:
print(f'Training Data Located at: \n\t- {player_train_dir} \n\t- {nonplayer_train_dir} \n')
print(f'Testing Data Located at: \n\t- {player_test_dir} \n\t- {nonplayer_test_dir} \n')

Training Data Located at: 
	- data/player/Blynde/train 
	- data/character/train 

Testing Data Located at: 
	- data/player/Blynde/test 
	- data/character/test 



# Testing

In [16]:
# testing data
data_test_one_round = player_data(player_test_dir,
                                  nonplayer_test_dir,
                                  batch_size = 32,
                                  repeat = False)

score = model.evaluate(data_test_one_round, verbose=1)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

20/20 [==============================] - 1s 41ms/step - loss: 0.2771 - binary_accuracy: 0.8844

Test score: 0.277
Test accuracy: 88%


# Confusion Matrix

In [17]:
import seaborn as sn
import pandas as pd
from sklearn.metrics import confusion_matrix

In [18]:
num_batches = 1
data_conf = player_data(player_test_dir,
                        nonplayer_test_dir,
                        batch_size = 25,
                        repeat=False)

In [19]:
batch_preds = []
batch_labels = []
i = 0

for xi, yi in data_conf:
    batch_preds.append(np.around(model.predict(xi)).astype(int).reshape((-1)))
    batch_labels.append(yi)
    i += 1
    if i > 500:
        break
    
#     i+=1
#     display_progress(i, num_batches)
# display_progress(num_batches, num_batches)
    
pred = np.concatenate(batch_preds)
labels = np.concatenate(batch_labels)

conf_matrix = confusion_matrix(labels, pred, normalize='all')[::-1, ::-1] # reverse so true positive is top left, true negative is bottom right
conf_matrix = np.around(conf_matrix, 3)
conf_df = pd.DataFrame(conf_matrix, index=['player', 'nonplayer'], columns=['player', 'nonplayer'])

In [20]:
conf_df

,player,nonplayer
player,0.461,0.059
nonplayer,0.065,0.415


# Training

In [8]:
# training data
small_batches = player_data(player_train_dir,
                            nonplayer_train_dir,
                            repeat = True,
                            batch_size = 32)

# training data with large batches
big_batches = player_data(player_train_dir,
                          nonplayer_train_dir,
                          repeat = True,
                          batch_size = 128)

# testing data
data_test = player_data(player_test_dir,
                        nonplayer_test_dir,
                        repeat = True,
                        batch_size = 25)

In [9]:
# Training Loop
model.fit(small_batches,
          epochs=10,
          steps_per_epoch=300,
          verbose=1)

score = model.evaluate(data_test, verbose=0, steps=100)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

Epoch 1/10
300/300 [==============================] - 14s 45ms/step - loss: 0.5024 - binary_accuracy: 0.7534
Epoch 2/10
300/300 [==============================] - 13s 44ms/step - loss: 0.4031 - binary_accuracy: 0.8161
Epoch 3/10
300/300 [==============================] - 13s 44ms/step - loss: 0.3734 - binary_accuracy: 0.8336
Epoch 4/10
300/300 [==============================] - 13s 44ms/step - loss: 0.3577 - binary_accuracy: 0.8431
Epoch 5/10
300/300 [==============================] - 13s 44ms/step - loss: 0.3432 - binary_accuracy: 0.8492
Epoch 6/10
300/300 [==============================] - 13s 44ms/step - loss: 0.3294 - binary_accuracy: 0.8552
Epoch 7/10
300/300 [==============================] - 13s 44ms/step - loss: 0.3381 - binary_accuracy: 0.8503
Epoch 8/10
300/300 [==============================] - 13s 44ms/step - loss: 0.3236 - binary_accuracy: 0.8584
Epoch 9/10
300/300 [==============================] - 13s 45ms/step - loss: 0.3060 - binary_accuracy: 0.8678
Epoch 10/10
300/300

In [10]:
# Training Loop
model.fit(big_batches,
          epochs=5,
          steps_per_epoch=75,
          verbose=1)

score = model.evaluate(data_test, verbose=0, steps=100)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

Epoch 1/5
75/75 [==============================] - 14s 188ms/step - loss: 0.3148 - binary_accuracy: 0.8629
Epoch 2/5
75/75 [==============================] - 14s 185ms/step - loss: 0.2950 - binary_accuracy: 0.8735
Epoch 3/5
75/75 [==============================] - 14s 184ms/step - loss: 0.2784 - binary_accuracy: 0.8819
Epoch 4/5
75/75 [==============================] - 14s 186ms/step - loss: 0.2930 - binary_accuracy: 0.8776
Epoch 5/5
75/75 [==============================] - 14s 181ms/step - loss: 0.2883 - binary_accuracy: 0.8773

Test score: 0.288
Test accuracy: 88%
